## run_FFM_hp_search_ADD.ipynb Code 실행 결과 정리

In [103]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Thu Dec  8 11:41:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   35C    P0    38W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

## 1. ADD 
----------

ADD, Resmax random serach하는데 4일 걸림

In [85]:
import numpy as np

import os
import pandas as pd
import pickle
#from models.BCresnet2 import get_BCResNet, get_inputshape
from sklearn.metrics import roc_curve
from utils.DataLoader import data_loader
from utils.Generator0 import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname


reslist = ['res_mask_ADD_abal/'+fnm for fnm in os.listdir('res_mask_ADD_abal/')]

data = []
allkeys = []
for i in range(len(reslist)) :
    with open(reslist[i], 'rb') as f:
        data1 = pickle.load(f)
    data.append(data1)
    allkeys += data1.keys()

allkeys = list(set(allkeys))

newlist = {}
for i in range(len(data)) :
    if i == 0 :
        for k1 in allkeys :
            try :
                newlist[k1] = [ data[i][k1] ]
            except :
                newlist[k1] = [-999]
    else :
        for k1 in allkeys :
            try :
                newlist[k1].append(data[i][k1])
            except :
                newlist[k1].append(-999)

df1 = pd.DataFrame(newlist)
df1.eer_val = pd.to_numeric(df1.eer_val)
df1.eer_eval = pd.to_numeric(df1.eer_eval)


In [86]:
len(df1)

758

In [87]:
df1.keys()

Index(['sr', 'shuffle', 'sec', 'tnow', 'model', 'filter_scale', 'tofile',
       'beta_param', 'eer_eval', 'lowpass', 'feature', 'n_classes', 'data_dir',
       'specmix', 'dropout_rate', 'saved_model', 'cutout', 'eer_val', 'n_bins',
       'cutmix', 'fmin', 'human_weight', 'highpass', 'specaug', 'batch_size',
       'filteraug', 'ranfilter2'],
      dtype='object')

### filter aug

In [88]:
len(df1[df1.filteraug!= -999])

235

In [89]:
df1[df1.filteraug!= -999][['model','filteraug','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

,model,filteraug,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
707,BCResMax_ADD_,"[[2, 5], 2, linear, [-6, 6]]",False,False,False,False,False,9.6105,0.1580
602,BCResMax_ADD_,"[[3, 6], 8, step, [-1, 1]]",False,False,False,False,False,0.0000,0.1654
601,BCResMax_ADD_,"[[3, 6], 2, step, [-1, 1]]",False,False,False,False,False,0.0002,0.1734
698,BCResMax_ADD_,"[[2, 5], 10, linear, [-1, 1]]",False,False,False,False,False,0.0005,0.1744
703,BCResMax_ADD_,"[[2, 5], 2, linear, [-1, 1]]",False,False,False,False,False,1.9221,0.1764
...,...,...,...,...,...,...,...,...,...
563,ResMax_ADD_,"[[5, 8], 6, step, [-1, 1]]",False,False,False,False,False,0.0002,0.2598
649,ResMax_ADD_,"[[3, 6], 10, linear, [-1, 1]]",False,False,False,False,False,0.0003,0.2704
566,ResMax_ADD_,"[[5, 8], 10, step, [-1, 1]]",False,False,False,False,False,0.0004,0.2719
526,ResMax_ADD_,"[[2, 5], 4, step, [-1, 1]]",False,False,False,False,False,7.6884,0.2731


### 1. filteraug(step) - ResMax

In [91]:
df1[df1.model == "ResMax_ADD_"][df1.filteraug!= -999][['model','filteraug','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,filteraug,beta_param,eer_val,eer_eval
662,ResMax_ADD_,"[[5, 8], 8, linear, [-10, 10]]",False,3.8442,0.1433
544,ResMax_ADD_,"[[2, 5], 4, step, [-6, 6]]",False,0.0000,0.1580
525,ResMax_ADD_,"[[2, 5], 6, step, [-10, 10]]",False,0.0000,0.1626
561,ResMax_ADD_,"[[3, 6], 6, step, [-10, 10]]",False,0.0004,0.1691
552,ResMax_ADD_,"[[2, 5], 10, step, [-6, 6]]",False,3.8442,0.1706
...,...,...,...,...,...
563,ResMax_ADD_,"[[5, 8], 6, step, [-1, 1]]",False,0.0002,0.2598
649,ResMax_ADD_,"[[3, 6], 10, linear, [-1, 1]]",False,0.0003,0.2704
566,ResMax_ADD_,"[[5, 8], 10, step, [-1, 1]]",False,0.0004,0.2719
526,ResMax_ADD_,"[[2, 5], 4, step, [-1, 1]]",False,7.6884,0.2731


### 2. filteraug(step) - LCNN

In [92]:
df1[df1.model == "LCNN_ADD_"][df1.filteraug!= -999][['model','filteraug','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,filteraug,beta_param,eer_val,eer_eval
580,LCNN_ADD_,"[[3, 6], 8, step, [-6, 6]]",False,0.0000,0.1604
585,LCNN_ADD_,"[[5, 8], 10, step, [-10, 10]]",False,0.0000,0.1636
581,LCNN_ADD_,"[[3, 6], 4, step, [-6, 6]]",False,0.0000,0.1674
584,LCNN_ADD_,"[[5, 8], 4, step, [-10, 10]]",False,3.8442,0.1713
680,LCNN_ADD_,"[[3, 6], 2, linear, [-10, 10]]",False,9.6105,0.1723
579,LCNN_ADD_,"[[3, 6], 6, step, [-6, 6]]",False,0.0000,0.1737
586,LCNN_ADD_,"[[5, 8], 4, step, [-6, 6]]",False,1.9221,0.1773
582,LCNN_ADD_,"[[3, 6], 8, step, [-6, 6]]",False,0.0000,0.1838
691,LCNN_ADD_,"[[5, 8], 4, linear, [-10, 10]]",False,1.9221,0.1853
676,LCNN_ADD_,"[[2, 5], 4, linear, [-1, 1]]",False,0.0008,0.1916


### 3. filteraug(step) - BCResMax

In [93]:
df1[df1.model == "BCResMax_ADD_"][df1.filteraug!= -999][['model','filteraug','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,filteraug,beta_param,eer_val,eer_eval
707,BCResMax_ADD_,"[[2, 5], 2, linear, [-6, 6]]",False,9.6105,0.1580
602,BCResMax_ADD_,"[[3, 6], 8, step, [-1, 1]]",False,0.0000,0.1654
601,BCResMax_ADD_,"[[3, 6], 2, step, [-1, 1]]",False,0.0002,0.1734
698,BCResMax_ADD_,"[[2, 5], 10, linear, [-1, 1]]",False,0.0005,0.1744
703,BCResMax_ADD_,"[[2, 5], 2, linear, [-1, 1]]",False,1.9221,0.1764
598,BCResMax_ADD_,"[[3, 6], 6, step, [-1, 1]]",False,3.8442,0.1783
605,BCResMax_ADD_,"[[5, 8], 10, step, [-10, 10]]",False,0.0004,0.1846
709,BCResMax_ADD_,"[[3, 6], 4, linear, [-1, 1]]",False,0.0006,0.1864
599,BCResMax_ADD_,"[[3, 6], 6, step, [-1, 1]]",False,0.0004,0.1891
616,BCResMax_ADD_,"[[5, 8], 10, linear, [-1, 1]]",False,0.0006,0.1901


### 4. filteraug(step) - DDWS

In [94]:
df1[df1.model == "DDWSseq_ADD_"][df1.filteraug!= -999][['model','filteraug','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,filteraug,beta_param,eer_val,eer_eval
621,DDWSseq_ADD_,"[[3, 6], 10, step, [-10, 10]]",False,0.0005,0.1269
618,DDWSseq_ADD_,"[[3, 6], 2, step, [-10, 10]]",False,9.6105,0.1394
622,DDWSseq_ADD_,"[[3, 6], 10, step, [-10, 10]]",False,0.0004,0.1409
636,DDWSseq_ADD_,"[[5, 8], 8, linear, [-6, 6]]",False,0.0003,0.1464
624,DDWSseq_ADD_,"[[5, 8], 10, step, [-10, 10]]",False,0.0004,0.1597
637,DDWSseq_ADD_,"[[5, 8], 6, linear, [-10, 10]]",False,0.0003,0.1619
735,DDWSseq_ADD_,"[[2, 5], 2, linear, [-6, 6]]",False,3.8442,0.1619
746,DDWSseq_ADD_,"[[3, 6], 8, linear, [-6, 6]]",False,0.0004,0.1643
627,DDWSseq_ADD_,"[[5, 8], 10, step, [-6, 6]]",False,0.0002,0.1713
635,DDWSseq_ADD_,"[[5, 8], 6, linear, [-10, 10]]",False,0.0002,0.1720


### cutout

In [367]:
len(df1[df1.cutout!= False])

113

In [368]:
df1[df1.cutout!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
234,BCResMax_ADD_,0.1,False,False,False,False,5.7663,0.1834
244,BCResMax_ADD_,0.4,False,False,False,False,0.0004,0.1860
236,BCResMax_ADD_,0.2,False,False,False,False,9.6105,0.1869
235,BCResMax_ADD_,0.1,False,False,False,False,3.8442,0.1878
246,BCResMax_ADD_,0.5,False,False,False,False,0.0004,0.1900
...,...,...,...,...,...,...,...,...
23,ResMax_ADD_,0.8,False,False,False,False,0.0016,0.2321
20,ResMax_ADD_,0.7,False,False,False,False,0.0017,0.2331
16,ResMax_ADD_,0.5,False,False,False,False,0.0021,0.2332
9,ResMax_ADD_,0.3,False,False,False,False,0.0004,0.2342


In [405]:
df1[df1.cutout!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].groupby( ['model','cutout' ])['eer_eval'].agg(**{
    'min_eer_eval':'min',
    'mean_eer_eval':'mean',
    'std_eer_eval' :'std'
}).reset_index().sort_values('mean_eer_eval')

,model,cutout,min_eer_eval,mean_eer_eval,std_eer_eval
23,LCNN_ADD_,0.6,0.1549,0.177467,0.019571
18,LCNN_ADD_,0.1,0.1768,0.190450,0.008764
22,LCNN_ADD_,0.5,0.1751,0.191067,0.014721
20,LCNN_ADD_,0.3,0.1853,0.191567,0.008857
24,LCNN_ADD_,0.7,0.1807,0.192367,0.016942
0,BCResMax_ADD_,0.1,0.1834,0.193300,0.013517
26,LCNN_ADD_,0.9,0.1884,0.193800,0.005667
13,DDWSseq_ADD_,0.5,0.1643,0.196100,0.029618
21,LCNN_ADD_,0.4,0.1954,0.196200,0.001386
19,LCNN_ADD_,0.2,0.1807,0.196700,0.014871


#### cutout 최적 parameter  
* LCNN_ADD_  0.6  
* BCResMax_ADD_  0.1  
* DDWSseq_ADD_  0.5
* ResMax_ADD_  0.9

### cutmix

In [370]:
## ResMax_ADD_ cutmix 코드 오타로 다시 돌려야 함

In [406]:
len(df1[df1.cutmix!= False])

108

도커에서 ResMax_ADD_ 다시 돌림

In [407]:
df1[df1.model == "ResMax_ADD_"][df1.cutmix!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','cutmix'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
29,ResMax_ADD_,False,0.7,False,False,False,5.7663,0.1674
30,ResMax_ADD_,False,0.7,False,False,False,0.0000,0.1783
31,ResMax_ADD_,False,0.7,False,False,False,3.8442,0.1580
32,ResMax_ADD_,False,0.7,False,False,False,5.7663,0.1860
33,ResMax_ADD_,False,0.7,False,False,False,7.6884,0.1597
34,ResMax_ADD_,False,0.7,False,False,False,0.0005,0.1674
35,ResMax_ADD_,False,0.7,False,False,False,0.0000,0.1674
36,ResMax_ADD_,False,0.7,False,False,False,0.0002,0.1884
37,ResMax_ADD_,False,0.7,False,False,False,0.0000,0.1626
38,ResMax_ADD_,False,0.7,False,False,False,0.0000,0.1611


In [373]:
df1[df1.cutmix!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','cutmix'] )

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
260,BCResMax_ADD_,False,0.1,False,False,False,5.7663,0.1534
261,BCResMax_ADD_,False,0.1,False,False,False,0.0002,0.1643
262,BCResMax_ADD_,False,0.1,False,False,False,0.0008,0.1192
263,BCResMax_ADD_,False,0.2,False,False,False,1.9221,0.1363
264,BCResMax_ADD_,False,0.2,False,False,False,0.0002,0.2597
...,...,...,...,...,...,...,...,...
51,ResMax_ADD_,False,0.7,False,False,False,3.8442,0.1681
52,ResMax_ADD_,False,0.7,False,False,False,0.0000,0.1664
53,ResMax_ADD_,False,0.7,False,False,False,0.0003,0.1699
54,ResMax_ADD_,False,0.7,False,False,False,0.0002,0.1650


In [409]:
df1[df1.cutmix!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].groupby( ['model','cutmix'] )['eer_eval'].agg(**{
    'min_eer_eval':'min',
    'mean_eer_eval':'mean',
    'std_eer_eval' :'std'
}).reset_index().sort_values('mean_eer_eval')

,model,cutmix,min_eer_eval,mean_eer_eval,std_eer_eval
8,BCResMax_ADD_,0.9,0.1269,0.139833,0.012147
17,DDWSseq_ADD_,0.9,0.1332,0.143800,0.009374
0,BCResMax_ADD_,0.1,0.1192,0.145633,0.023532
13,DDWSseq_ADD_,0.5,0.1245,0.154633,0.026516
7,BCResMax_ADD_,0.8,0.1527,0.155500,0.004258
14,DDWSseq_ADD_,0.6,0.1324,0.156700,0.025248
19,LCNN_ADD_,0.2,0.1440,0.159867,0.014180
6,BCResMax_ADD_,0.7,0.1469,0.160800,0.012043
12,DDWSseq_ADD_,0.4,0.1402,0.160900,0.018335
16,DDWSseq_ADD_,0.8,0.1416,0.161167,0.028608


#### cutmix hp
* BCResMax_ADD_ 0.9
* DDWSseq_ADD_ 0.9
* ResMax_ADD_ 0.1 (도커)
* LCNN_ADD_ 0.2

### specaug

22/8/22 ddws specaug 부분 다시 돌려야 함(컴퓨터꺼짐 ㅠ)

In [410]:
len(df1[df1.specaug!= False])

152

In [423]:
len(df1[df1.specaug!= False][df1.model == "ResMax_ADD_"])

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


30

In [411]:
len(df1[df1.specaug!= False][df1.model == "DDWSseq_ADD_"])

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


29

In [467]:
len(df1[df1.specaug!= False][df1.model == "BCResMax_ADD_"])

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


63

In [426]:
df_left_join[df_left_join.specaug!= False][df_left_join.model == "DDWSseq_ADD_"][['model','cutout','cutmix','specaug','specmix','num_spec','beta_param','eer_val','eer_eval']].sort_values( ['model','num_spec'] )

,model,cutout,cutmix,specaug,specmix,num_spec,beta_param,eer_val,eer_eval
93,DDWSseq_ADD_,False,False,"[[1, 5], [1, 5]]",False,0,False,0.0003,0.3122
94,DDWSseq_ADD_,False,False,"[[1, 5], [1, 5]]",False,0,False,0.0013,0.2258
95,DDWSseq_ADD_,False,False,"[[1, 5], [1, 5]]",False,0,False,0.0002,0.2468
96,DDWSseq_ADD_,False,False,"[[1, 10], [1, 10]]",False,1,False,0.0005,0.2328
97,DDWSseq_ADD_,False,False,"[[1, 10], [1, 10]]",False,1,False,0.0003,0.2328
98,DDWSseq_ADD_,False,False,"[[1, 10], [1, 10]]",False,1,False,0.0004,0.2328
99,DDWSseq_ADD_,False,False,"[[1, 10], [1, 15]]",False,2,False,0.0005,0.2398
100,DDWSseq_ADD_,False,False,"[[1, 10], [1, 15]]",False,2,False,0.0007,0.2622
101,DDWSseq_ADD_,False,False,"[[1, 10], [1, 15]]",False,2,False,0.0010,0.2335
102,DDWSseq_ADD_,False,False,"[[1, 15], [1, 10]]",False,3,False,0.0009,0.2118


In [413]:
len(df1[df1.specaug!= False][df1.model == "LCNN_ADD_"])

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


30

In [379]:
df1[df1.specaug!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
317,BCResMax_ADD_,False,False,"[[2, 15], [2, 15]]",False,False,1.9221,0.1604
292,BCResMax_ADD_,False,False,"[[1, 5], [1, 5]]",False,False,0.0000,0.1611
290,BCResMax_ADD_,False,False,"[[1, 5], [1, 5]]",False,False,0.0002,0.1723
458,BCResMax_ADD_,False,False,"[[2, 15], [2, 10]]",False,False,0.0003,0.1783
425,BCResMax_ADD_,False,False,"[[1, 10], [1, 10]]",False,False,0.0000,0.1790
...,...,...,...,...,...,...,...,...
67,ResMax_ADD_,False,False,"[[1, 15], [1, 10]]",False,False,0.0008,0.2195
85,ResMax_ADD_,False,False,"[[2, 15], [2, 15]]",False,False,0.0003,0.2208
57,ResMax_ADD_,False,False,"[[1, 5], [1, 5]]",False,False,0.0002,0.2227
80,ResMax_ADD_,False,False,"[[2, 15], [2, 10]]",False,False,0.0002,0.2244


In [472]:
spec_list = []
for i, row in df1[df1.specaug!=False].iterrows():
    run_scored = row['specaug']
    spec_list.append(run_scored)
    
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]

result = []
for i in spec_list:
    result.append(spec_hps.index(i))
    
df_result = pd.DataFrame(result, columns = ['num_spec'])
df3 = df1[df1.specaug != False].reset_index()
df_left_join = pd.merge(df3, df_result, left_index = True, right_index = True, how = 'left')

In [473]:
df_left_join[df_left_join.model == "BCResMax_ADD_"]

,index,data_dir,human_weight,tnow,n_bins,ranfilter2,sr,cutmix,saved_model,model,...,eer_val,fmin,eer_eval,cutout,sec,batch_size,n_classes,tofile,feature,num_spec
60,287,/home/ubuntu/data/ADD/,5.0,2022-08-11 15:04:42.891300,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,3.8442,5,0.2094,False,9.0,16,2,tmp,cqt,0
61,288,/home/ubuntu/data/ADD/,5.0,2022-08-11 15:53:06.424322,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,0.0004,5,0.2702,False,9.0,16,2,tmp,cqt,0
62,289,/home/ubuntu/data/ADD/,5.0,2022-08-11 16:43:48.585386,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,0.0000,5,0.2087,False,9.0,16,2,tmp,cqt,0
63,290,/home/ubuntu/data/ADD/,5.0,2022-08-16 09:38:28.354895,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,0.0002,5,0.1723,False,9.0,16,2,tmp,cqt,0
64,291,/home/ubuntu/data/ADD/,5.0,2022-08-16 10:26:58.511368,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,0.0012,5,0.2038,False,9.0,16,2,tmp,cqt,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,458,/home/ubuntu/data/ADD/,5.0,2022-08-23 06:43:37.577708,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,0.0003,5,0.1783,False,9.0,16,2,tmp,cqt,8
148,459,/home/ubuntu/data/ADD/,5.0,2022-08-23 07:32:20.594915,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,5.7663,5,0.2437,False,9.0,16,2,tmp,cqt,8
149,460,/home/ubuntu/data/ADD/,5.0,2022-08-23 08:21:22.691062,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,5.7663,5,0.2531,False,9.0,16,2,tmp,cqt,9
150,461,/home/ubuntu/data/ADD/,5.0,2022-08-23 09:10:03.445161,100,False,16000,False,saved_models/model_name!BCResMax_ADD_!dropout_...,BCResMax_ADD_,...,0.0002,5,0.1814,False,9.0,16,2,tmp,cqt,9


In [474]:
df_left_join[df_left_join.model == "BCResMax_ADD_"][['model','cutout','cutmix','specaug','specmix','num_spec','beta_param','eer_val','eer_eval']].sort_values('num_spec')

,model,cutout,cutmix,specaug,specmix,num_spec,beta_param,eer_val,eer_eval
60,BCResMax_ADD_,False,False,"[[1, 5], [1, 5]]",False,0,False,3.8442,0.2094
112,BCResMax_ADD_,False,False,"[[1, 5], [1, 5]]",False,0,False,5.7663,0.2164
111,BCResMax_ADD_,False,False,"[[1, 5], [1, 5]]",False,0,False,0.0003,0.2350
65,BCResMax_ADD_,False,False,"[[1, 5], [1, 5]]",False,0,False,0.0000,0.1611
110,BCResMax_ADD_,False,False,"[[1, 5], [1, 5]]",False,0,False,0.0008,0.2157
...,...,...,...,...,...,...,...,...,...
91,BCResMax_ADD_,False,False,"[[2, 15], [2, 15]]",False,9,False,0.0002,0.2664
150,BCResMax_ADD_,False,False,"[[2, 15], [2, 15]]",False,9,False,0.0002,0.1814
90,BCResMax_ADD_,False,False,"[[2, 15], [2, 15]]",False,9,False,1.9221,0.1604
92,BCResMax_ADD_,False,False,"[[2, 15], [2, 15]]",False,9,False,0.0005,0.1853


In [428]:
df_left_join[['model','cutout','cutmix','specaug','specmix','num_spec','beta_param','eer_val','eer_eval']].groupby(['model','num_spec'])['eer_eval'].agg(**{
    'min_eer_eval':'min',
    'mean_eer_eval':'mean',
    'std_eer_eval' :'std'
}).reset_index().sort_values('mean_eer_eval')

,model,num_spec,min_eer_eval,mean_eer_eval,std_eer_eval
32,ResMax_ADD_,2,0.1814,0.194100,0.015805
21,LCNN_ADD_,1,0.1884,0.197267,0.008045
35,ResMax_ADD_,5,0.1978,0.200867,0.002656
22,LCNN_ADD_,2,0.1923,0.200867,0.010443
36,ResMax_ADD_,6,0.1908,0.202733,0.013970
37,ResMax_ADD_,7,0.1930,0.203700,0.009273
29,LCNN_ADD_,9,0.2014,0.203933,0.004131
23,LCNN_ADD_,3,0.2017,0.206033,0.003940
34,ResMax_ADD_,4,0.2055,0.207033,0.002656
28,LCNN_ADD_,8,0.2017,0.208233,0.006313


#### specaug hp
* ResMax_ADD_	 [[1, 10], [1, 15]]
* LCNN_ADD_   [[1, 10], [1, 10]]
* BCResMax_ADD_ [[2, 15], [2, 15]]
* DDWSseq_ADD_	 [[2, 15], [2, 10]]

### specmix


In [431]:
len(df1[df1.specmix!= False])

150

In [476]:
len(df1[df1.specmix!= False][df1.model == "BCResMax_ADD_"])

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


60

In [475]:
len(df1[df1.specmix!= False][df1.model == "DDWSseq_ADD_"])

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


30

In [432]:
df1[df1.specmix!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
491,BCResMax_ADD_,False,False,False,"[[2, 15], [2, 15]]",False,0.6302,0.3226
349,BCResMax_ADD_,False,False,False,"[[2, 15], [2, 15]]",False,0.4862,0.3496
481,BCResMax_ADD_,False,False,False,"[[2, 10], [2, 10]]",False,0.5640,0.4003
344,BCResMax_ADD_,False,False,False,"[[2, 15], [2, 10]]",False,0.8543,0.4066
335,BCResMax_ADD_,False,False,False,"[[2, 5], [2, 5]]",False,0.8812,0.4153
...,...,...,...,...,...,...,...,...
99,ResMax_ADD_,False,False,False,"[[1, 15], [1, 15]]",False,0.9976,0.7374
98,ResMax_ADD_,False,False,False,"[[1, 15], [1, 15]]",False,0.9964,0.7531
95,ResMax_ADD_,False,False,False,"[[1, 15], [1, 10]]",False,0.9994,0.7577
91,ResMax_ADD_,False,False,False,"[[1, 10], [1, 10]]",False,0.9978,0.7671


In [437]:
spec_list = []
for i, row in df1[df1.specmix!=False].iterrows():
    run_scored = row['specmix']
    spec_list.append(run_scored)
    
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]

result = []
for i in spec_list:
    result.append(spec_hps.index(i))
    
df_result = pd.DataFrame(result, columns = ['num_spec'])
df3 = df1[df1.specmix != False].reset_index()
df_left_join = pd.merge(df3, df_result, left_index = True, right_index = True, how = 'left')
df_left_join[['model', 'cutout','cutmix','specaug','specmix','num_spec','beta_param','eer_val','eer_eval']]

,model,cutout,cutmix,specaug,specmix,num_spec,beta_param,eer_val,eer_eval
0,ResMax_ADD_,False,False,False,"[[1, 5], [1, 5]]",0,False,0.8737,0.4828
1,ResMax_ADD_,False,False,False,"[[1, 5], [1, 5]]",0,False,0.8752,0.5878
2,ResMax_ADD_,False,False,False,"[[1, 5], [1, 5]]",0,False,0.9114,0.5521
3,ResMax_ADD_,False,False,False,"[[1, 10], [1, 10]]",1,False,0.9991,0.7198
4,ResMax_ADD_,False,False,False,"[[1, 10], [1, 10]]",1,False,0.9970,0.6961
...,...,...,...,...,...,...,...,...,...
145,DDWSseq_ADD_,False,False,False,"[[2, 15], [2, 10]]",8,False,0.5668,0.3083
146,DDWSseq_ADD_,False,False,False,"[[2, 15], [2, 10]]",8,False,0.8620,0.3986
147,DDWSseq_ADD_,False,False,False,"[[2, 15], [2, 15]]",9,False,0.7363,0.5209
148,DDWSseq_ADD_,False,False,False,"[[2, 15], [2, 15]]",9,False,0.6727,0.6122


In [435]:
df_left_join[['model','cutout','cutmix','specaug','specmix','num_spec','beta_param','eer_val','eer_eval']].groupby(['model','num_spec'])['eer_eval'].agg(**{
    'min_eer_eval':'min',
    'mean_eer_eval':'mean',
    'std_eer_eval' :'std'
}).reset_index().sort_values('mean_eer_eval')

,model,num_spec,min_eer_eval,mean_eer_eval,std_eer_eval
17,DDWSseq_ADD_,7,0.2944,0.359767,0.089521
18,DDWSseq_ADD_,8,0.3083,0.398100,0.089551
14,DDWSseq_ADD_,4,0.3045,0.474200,0.148936
9,BCResMax_ADD_,9,0.3226,0.476767,0.122038
19,DDWSseq_ADD_,9,0.3192,0.484100,0.149926
26,LCNN_ADD_,6,0.4517,0.495867,0.064191
27,LCNN_ADD_,7,0.4650,0.503667,0.046456
8,BCResMax_ADD_,8,0.4066,0.507233,0.057854
29,LCNN_ADD_,9,0.4408,0.513700,0.084690
15,DDWSseq_ADD_,5,0.3902,0.522567,0.171047


#### specmix hp
* DDWSseq_ADD_ 7번
* BCResMax_ADD_ 9번
* LCNN_ADD_ 6번 
* ResMax 0번

## 가장 성능이 좋은 모델 확인

In [438]:
df1[df1.eer_eval <= 0.17][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values('eer_eval')

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
262,BCResMax_ADD_,False,0.1,False,False,False,0.0008,0.1192
389,DDWSseq_ADD_,False,0.5,False,False,False,0.0002,0.1245
285,BCResMax_ADD_,False,0.9,False,False,False,0.0003,0.1269
267,BCResMax_ADD_,False,0.3,False,False,False,0.0003,0.1323
392,DDWSseq_ADD_,False,0.6,False,False,False,5.7663,0.1324
...,...,...,...,...,...,...,...,...
43,ResMax_ADD_,False,0.7,False,False,False,0.0000,0.1689
271,BCResMax_ADD_,False,0.4,False,False,False,5.7663,0.1689
396,DDWSseq_ADD_,False,0.7,False,False,False,0.0003,0.1691
160,LCNN_ADD_,False,0.5,False,False,False,0.0007,0.1696


## 2. ASV
----------------

In [95]:
import numpy as np

import os
import pandas as pd
import pickle
#from models.BCresnet2 import get_BCResNet, get_inputshape
from sklearn.metrics import roc_curve
from utils.DataLoader import data_loader
from utils.Generator0 import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname


reslist = ['res_mask_LA_ab_hp/'+fnm for fnm in os.listdir('res_mask_LA_ab_hp/')]

data = []
allkeys = []
for i in range(len(reslist)) :
    with open(reslist[i], 'rb') as f:
        data1 = pickle.load(f)
    data.append(data1)
    allkeys += data1.keys()

allkeys = list(set(allkeys))

newlist = {}
for i in range(len(data)) :
    if i == 0 :
        for k1 in allkeys :
            try :
                newlist[k1] = [ data[i][k1] ]
            except :
                newlist[k1] = [-999]
    else :
        for k1 in allkeys :
            try :
                newlist[k1].append(data[i][k1])
            except :
                newlist[k1].append(-999)

df1 = pd.DataFrame(newlist)
df1.eer_val = pd.to_numeric(df1.eer_val)
df1.eer_eval = pd.to_numeric(df1.eer_eval)


### filter aug

In [96]:
len(df1[df1.filteraug!= -999])

299

In [97]:
df1[df1.filteraug!= -999][['model','filteraug','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

,model,filteraug,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
511,BCResMax_LA_,"[[3, 6], 2, linear, [-6, 6]]",False,False,False,False,False,0.0055,0.0227
526,BCResMax_LA_,"[[5, 8], 2, linear, [-1, 1]]",False,False,False,False,False,0.0054,0.0239
499,BCResMax_LA_,"[[2, 5], 10, linear, [-6, 6]]",False,False,False,False,False,0.0084,0.0240
388,BCResMax_LA_,"[[5, 8], 8, step, [-1, 1]]",False,False,False,False,False,0.0043,0.0243
500,BCResMax_LA_,"[[2, 5], 4, linear, [-6, 6]]",False,False,False,False,False,0.0042,0.0244
...,...,...,...,...,...,...,...,...,...
460,ResMax_LA_,"[[5, 8], 10, linear, [-1, 1]]",False,False,False,False,False,0.0082,0.0398
333,ResMax_LA_,"[[2, 5], 6, linear, [-1, 1]]",False,False,False,False,False,0.0072,0.0401
260,ResMax_LA_,"[[2, 5], 10, step, [-10, 10]]",False,False,False,False,False,0.0116,0.0431
448,ResMax_LA_,"[[3, 6], 6, linear, [-1, 1]]",False,False,False,False,False,0.0122,0.0436


### 1. filteraug(step) - ResMax

In [98]:
a = df1[df1.model == "ResMax_LA_"][df1.filteraug!= -999 ][['model','filteraug','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )
# a.to_csv("filteraug(step)_resmax_la")
a

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,filteraug,beta_param,eer_val,eer_eval
268,ResMax_LA_,"[[2, 5], 2, step, [-1, 1]]",False,0.0047,0.0210
438,ResMax_LA_,"[[2, 5], 4, linear, [-6, 6]]",False,0.0055,0.0234
445,ResMax_LA_,"[[2, 5], 6, linear, [-6, 6]]",False,0.0059,0.0254
441,ResMax_LA_,"[[2, 5], 10, linear, [-6, 6]]",False,0.0049,0.0258
446,ResMax_LA_,"[[2, 5], 4, linear, [-6, 6]]",False,0.0074,0.0260
...,...,...,...,...,...
460,ResMax_LA_,"[[5, 8], 10, linear, [-1, 1]]",False,0.0082,0.0398
333,ResMax_LA_,"[[2, 5], 6, linear, [-1, 1]]",False,0.0072,0.0401
260,ResMax_LA_,"[[2, 5], 10, step, [-10, 10]]",False,0.0116,0.0431
448,ResMax_LA_,"[[3, 6], 6, linear, [-1, 1]]",False,0.0122,0.0436


### 2. filteraug(step) - LCNN

In [99]:
df1[df1.model == "LCNN_LA_"][df1.filteraug!= -999][['model','filteraug','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,filteraug,beta_param,eer_val,eer_eval
483,LCNN_LA_,"[[3, 6], 2, linear, [-6, 6]]",False,0.0038,0.0218
479,LCNN_LA_,"[[3, 6], 6, linear, [-1, 1]]",False,0.0034,0.0231
481,LCNN_LA_,"[[3, 6], 6, linear, [-6, 6]]",False,0.0046,0.0235
474,LCNN_LA_,"[[2, 5], 4, linear, [-1, 1]]",False,0.0048,0.0241
362,LCNN_LA_,"[[5, 8], 2, step, [-10, 10]]",False,0.0054,0.0243
351,LCNN_LA_,"[[2, 5], 6, step, [-1, 1]]",False,0.0044,0.0246
369,LCNN_LA_,"[[3, 6], 2, linear, [-6, 6]]",False,0.0043,0.0246
360,LCNN_LA_,"[[5, 8], 6, step, [-1, 1]]",False,0.0027,0.0248
485,LCNN_LA_,"[[3, 6], 2, linear, [-6, 6]]",False,0.0065,0.0249
367,LCNN_LA_,"[[2, 5], 6, linear, [-10, 10]]",False,0.0043,0.0252


### 3. filteraug(step) - BCResMax

In [100]:
df1[df1.model == "BCResMax_LA_"][df1.filteraug!= -999][['model','filteraug','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,filteraug,beta_param,eer_val,eer_eval
511,BCResMax_LA_,"[[3, 6], 2, linear, [-6, 6]]",False,0.0055,0.0227
526,BCResMax_LA_,"[[5, 8], 2, linear, [-1, 1]]",False,0.0054,0.0239
499,BCResMax_LA_,"[[2, 5], 10, linear, [-6, 6]]",False,0.0084,0.0240
388,BCResMax_LA_,"[[5, 8], 8, step, [-1, 1]]",False,0.0043,0.0243
500,BCResMax_LA_,"[[2, 5], 4, linear, [-6, 6]]",False,0.0042,0.0244
513,BCResMax_LA_,"[[3, 6], 2, linear, [-1, 1]]",False,0.0051,0.0245
394,BCResMax_LA_,"[[2, 5], 6, linear, [-1, 1]]",False,0.0039,0.0254
386,BCResMax_LA_,"[[3, 6], 10, step, [-10, 10]]",False,0.0094,0.0260
378,BCResMax_LA_,"[[2, 5], 2, step, [-6, 6]]",False,0.0060,0.0270
402,BCResMax_LA_,"[[3, 6], 2, linear, [-1, 1]]",False,0.0048,0.0275


### 4. filteraug(step) - DDWS

In [101]:
df1[df1.model == "DDWSseq_LA_"][df1.filteraug!= -999][['model','filteraug','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,filteraug,beta_param,eer_val,eer_eval
533,DDWSseq_LA_,"[[2, 5], 4, linear, [-1, 1]]",False,0.0037,0.0194
416,DDWSseq_LA_,"[[3, 6], 10, step, [-1, 1]]",False,0.0048,0.0205
423,DDWSseq_LA_,"[[2, 5], 10, linear, [-1, 1]]",False,0.0032,0.0213
534,DDWSseq_LA_,"[[2, 5], 2, linear, [-1, 1]]",False,0.0048,0.0214
437,DDWSseq_LA_,"[[5, 8], 8, linear, [-1, 1]]",False,0.0030,0.0216
537,DDWSseq_LA_,"[[2, 5], 2, linear, [-1, 1]]",False,0.0044,0.0227
413,DDWSseq_LA_,"[[3, 6], 10, step, [-1, 1]]",False,0.0042,0.0241
554,DDWSseq_LA_,"[[5, 8], 8, linear, [-1, 1]]",False,0.0041,0.0244
412,DDWSseq_LA_,"[[2, 5], 2, step, [-1, 1]]",False,0.0053,0.0245
528,DDWSseq_LA_,"[[2, 5], 6, linear, [-6, 6]]",False,0.0056,0.0245


### cutout

In [440]:
len(df1[df1.cutout!= False])

81

In [441]:
df1[df1.cutout!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
244,BCResMax_LA_,0.7,False,False,False,False,0.0034,0.0236
246,BCResMax_LA_,0.7,False,False,False,False,0.0058,0.0254
236,BCResMax_LA_,0.4,False,False,False,False,0.0062,0.0258
242,BCResMax_LA_,0.6,False,False,False,False,0.0066,0.0273
251,BCResMax_LA_,0.9,False,False,False,False,0.0044,0.0277
...,...,...,...,...,...,...,...,...
15,ResMax_LA_,0.6,False,False,False,False,0.0069,0.0478
8,ResMax_LA_,0.3,False,False,False,False,0.0075,0.0492
0,ResMax_LA_,0.1,False,False,False,False,0.0101,0.0509
10,ResMax_LA_,0.4,False,False,False,False,0.0075,0.0518


In [446]:
df1[df1.cutout!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].groupby( ['model', 'cutout'] )['eer_eval'].agg(**{
    'min_eer_eval':'min',
    'mean_eer_eval':'mean',
    'std_eer_eval' :'std'
}).reset_index().sort_values('mean_eer_eval')

,model,cutout,min_eer_eval,mean_eer_eval,std_eer_eval
6,BCResMax_LA_,0.7,0.0236,0.029900,0.009396
14,LCNN_LA_,0.6,0.0274,0.030600,0.003601
15,LCNN_LA_,0.7,0.0288,0.030700,0.001769
2,BCResMax_LA_,0.3,0.0299,0.031000,0.001905
10,LCNN_LA_,0.2,0.0301,0.031167,0.001361
8,BCResMax_LA_,0.9,0.0277,0.031367,0.006351
12,LCNN_LA_,0.4,0.0235,0.031633,0.007202
9,LCNN_LA_,0.1,0.0264,0.031667,0.004631
16,LCNN_LA_,0.8,0.0278,0.031767,0.003496
5,BCResMax_LA_,0.6,0.0273,0.032533,0.004934


#### cutout hp
* DDWS 0.7  (도커)
* BcResMax 0.6 (도커, 최고 성능이 너무 다름. 둘다 상위권인 0.6으로 선택)
* LCNN_LA_ 0.6
* ResMax_LA_ 0.2


### cutmix

In [447]:
len(df1[df1.cutmix!= False])

58

In [450]:
df1[df1.cutmix!= False][df1.model == "ResMax_LA_"][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','cutmix'] )

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
27,ResMax_LA_,False,0.1,False,False,False,0.0152,0.0353
28,ResMax_LA_,False,0.1,False,False,False,0.0145,0.0420
29,ResMax_LA_,False,0.1,False,False,False,0.0161,0.0458
30,ResMax_LA_,False,0.2,False,False,False,0.0152,0.0390
31,ResMax_LA_,False,0.2,False,False,False,0.0164,0.0296
32,ResMax_LA_,False,0.2,False,False,False,0.0131,0.0360
33,ResMax_LA_,False,0.3,False,False,False,0.0198,0.0382
34,ResMax_LA_,False,0.3,False,False,False,0.0175,0.0397
35,ResMax_LA_,False,0.3,False,False,False,0.0149,0.0354
36,ResMax_LA_,False,0.4,False,False,False,0.0141,0.0336


In [448]:
df1[df1.cutmix!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','cutmix'] )

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
253,BCResMax_LA_,False,0.1,False,False,False,0.0053,0.0276
254,BCResMax_LA_,False,0.1,False,False,False,0.0086,0.0289
255,BCResMax_LA_,False,0.1,False,False,False,0.0074,0.0245
256,BCResMax_LA_,False,0.2,False,False,False,0.0084,0.0312
139,LCNN_LA_,False,0.1,False,False,False,0.0068,0.0337
140,LCNN_LA_,False,0.1,False,False,False,0.0097,0.0355
141,LCNN_LA_,False,0.1,False,False,False,0.0101,0.0380
142,LCNN_LA_,False,0.2,False,False,False,0.0113,0.0461
143,LCNN_LA_,False,0.2,False,False,False,0.0105,0.0402
144,LCNN_LA_,False,0.2,False,False,False,0.0074,0.0411


In [452]:
df1[df1.cutmix!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].groupby(['model','cutmix'])['eer_eval'].agg(**{
    'min_eer_eval':'min',
    'mean_eer_eval':'mean',
    'std_eer_eval' :'std'
}).reset_index().sort_values('mean_eer_eval')

,model,cutmix,min_eer_eval,mean_eer_eval,std_eer_eval
0,BCResMax_LA_,0.1,0.0245,0.027000,0.002261
17,ResMax_LA_,0.7,0.0284,0.030467,0.002050
16,ResMax_LA_,0.6,0.0259,0.030633,0.004908
1,BCResMax_LA_,0.2,0.0312,0.031200,NaN
18,ResMax_LA_,0.8,0.0294,0.031867,0.002610
14,ResMax_LA_,0.4,0.0316,0.034333,0.003164
12,ResMax_LA_,0.2,0.0296,0.034867,0.004801
4,LCNN_LA_,0.3,0.0285,0.035567,0.007204
2,LCNN_LA_,0.1,0.0337,0.035733,0.002159
15,ResMax_LA_,0.5,0.0316,0.036467,0.004411


#### cutmix hp
* ResMax_LA_ 0.7
* LCNN_LA_ 0.3
* DDWS 0.2(도커)
* BcResMax 0.8(도커)


### specaug

In [453]:
len(df1[df1.specaug!= False])

60

In [393]:
df1[df1.specaug!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
174,LCNN_LA_,False,False,"[[1, 10], [1, 15]]",False,False,0.0026,0.0297
168,LCNN_LA_,False,False,"[[1, 5], [1, 5]]",False,False,0.0050,0.0312
186,LCNN_LA_,False,False,"[[2, 10], [2, 10]]",False,False,0.0043,0.0316
167,LCNN_LA_,False,False,"[[1, 5], [1, 5]]",False,False,0.0046,0.0317
175,LCNN_LA_,False,False,"[[1, 15], [1, 10]]",False,False,0.0054,0.0317
181,LCNN_LA_,False,False,"[[2, 5], [2, 5]]",False,False,0.0046,0.0323
183,LCNN_LA_,False,False,"[[2, 5], [2, 5]]",False,False,0.0058,0.0341
176,LCNN_LA_,False,False,"[[1, 15], [1, 10]]",False,False,0.0052,0.0350
185,LCNN_LA_,False,False,"[[2, 10], [2, 10]]",False,False,0.0048,0.0355
169,LCNN_LA_,False,False,"[[1, 10], [1, 10]]",False,False,0.0030,0.0361


In [454]:
spec_list = []
for i, row in df1[df1.specaug!=False].iterrows():
    run_scored = row['specaug']
    spec_list.append(run_scored)
    
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]

result = []
for i in spec_list:
    result.append(spec_hps.index(i))
    
df_result = pd.DataFrame(result, columns = ['num_spec'])
df3 = df1[df1.specaug != False].reset_index()
df_left_join = pd.merge(df3, df_result, left_index = True, right_index = True, how = 'left')

In [456]:
f = df_left_join[df_left_join.specaug!= False][['model','cutout','cutmix','specaug','num_spec','specmix','beta_param','eer_val','eer_eval']].groupby( ['model','num_spec'] )['eer_eval'].agg(**{
    'min_eer_eval':'min',
    'mean_eer_eval':'mean',
    'std_eer_eval' :'std'
}).reset_index()
f.sort_values('mean_eer_eval')

,model,num_spec,min_eer_eval,mean_eer_eval,std_eer_eval
0,LCNN_LA_,0,0.0312,0.034033,0.004481
5,LCNN_LA_,5,0.0323,0.034167,0.001901
3,LCNN_LA_,3,0.0317,0.034867,0.003102
6,LCNN_LA_,6,0.0316,0.035000,0.003180
2,LCNN_LA_,2,0.0297,0.037233,0.006854
1,LCNN_LA_,1,0.0361,0.038667,0.002223
15,ResMax_LA_,5,0.0369,0.039400,0.003579
4,LCNN_LA_,4,0.0378,0.040233,0.002940
7,LCNN_LA_,7,0.0394,0.040833,0.001358
8,LCNN_LA_,8,0.0365,0.041000,0.006300


#### specaug hp
* LCNN_LA_ [[1, 5], [1, 5]]0번
* ResMax_LA_ [[2, 5], [2, 5]]5번
* DDWS [[1, 10], [1, 15]]  (도커)
* BcResMax [[2, 5], [2, 5]]  (도커)

### specmix


In [457]:
df1[df1.specmix!= False][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values( ['model','eer_eval'] )

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
224,LCNN_LA_,False,False,False,"[[2, 15], [2, 15]]",False,0.3253,0.3517
221,LCNN_LA_,False,False,False,"[[2, 15], [2, 10]]",False,0.3523,0.3847
220,LCNN_LA_,False,False,False,"[[2, 15], [2, 10]]",False,0.3433,0.4155
217,LCNN_LA_,False,False,False,"[[2, 10], [2, 15]]",False,0.4650,0.4304
214,LCNN_LA_,False,False,False,"[[2, 10], [2, 10]]",False,0.5266,0.4437
223,LCNN_LA_,False,False,False,"[[2, 15], [2, 15]]",False,0.4518,0.4459
218,LCNN_LA_,False,False,False,"[[2, 10], [2, 15]]",False,0.4854,0.4576
219,LCNN_LA_,False,False,False,"[[2, 10], [2, 15]]",False,0.5019,0.4628
209,LCNN_LA_,False,False,False,"[[1, 15], [1, 15]]",False,0.6023,0.5105
225,LCNN_LA_,False,False,False,"[[2, 15], [2, 15]]",False,0.4975,0.5301


In [459]:
spec_list = []
for i, row in df1[df1.specmix!=False].iterrows():
    run_scored = row['specmix']
    spec_list.append(run_scored)
    
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]

result = []
for i in spec_list:
    result.append(spec_hps.index(i))
    
df_result = pd.DataFrame(result, columns = ['num_spec'])
df3 = df1[df1.specmix != False].reset_index()
df_left_join = pd.merge(df3, df_result, left_index = True, right_index = True, how = 'left')
df_left_join[['model', 'cutout','cutmix','specaug','specmix','num_spec','beta_param','eer_val','eer_eval']]

df_left_join[['model','cutout','cutmix','specaug','specmix','num_spec','beta_param','eer_val','eer_eval']].groupby(['model','num_spec'])['eer_eval'].agg(**{
    'min_eer_eval':'min',
    'mean_eer_eval':'mean',
    'std_eer_eval' :'std'
}).reset_index().sort_values('mean_eer_eval')

,model,num_spec,min_eer_eval,mean_eer_eval,std_eer_eval
18,ResMax_LA_,8,0.3817,0.416400,0.045866
9,LCNN_LA_,9,0.3517,0.442567,0.089247
7,LCNN_LA_,7,0.4304,0.450267,0.017400
19,ResMax_LA_,9,0.2618,0.452567,0.193714
17,ResMax_LA_,7,0.4385,0.482467,0.038877
8,LCNN_LA_,8,0.3847,0.518733,0.206055
6,LCNN_LA_,6,0.4437,0.523633,0.073267
16,ResMax_LA_,6,0.3993,0.528767,0.130820
4,LCNN_LA_,4,0.5105,0.567533,0.051297
13,ResMax_LA_,3,0.5971,0.628800,0.033472


#### specmix hp
* ResMax_LA_	[[2, 15], [2, 10]] 8번
* LCNN_LA_ [[2, 15], [2, 15]]9번
* DDWS [[2, 15], [2, 15]] (도커)
* BcResMax [[2, 15], [2, 10]] (도커)

## 가장 성능이 좋은 모델 확인

In [464]:
df1[df1.eer_eval <= 0.03][['model','cutout','cutmix','specaug','specmix','beta_param','eer_val','eer_eval']].sort_values('eer_eval')

,model,cutout,cutmix,specaug,specmix,beta_param,eer_val,eer_eval
123,LCNN_LA_,0.4,False,False,False,False,0.0039,0.0235
244,BCResMax_LA_,0.7,False,False,False,False,0.0034,0.0236
255,BCResMax_LA_,False,0.1,False,False,False,0.0074,0.0245
246,BCResMax_LA_,0.7,False,False,False,False,0.0058,0.0254
236,BCResMax_LA_,0.4,False,False,False,False,0.0062,0.0258
42,ResMax_LA_,False,0.6,False,False,False,0.0114,0.0259
114,LCNN_LA_,0.1,False,False,False,False,0.0058,0.0264
120,LCNN_LA_,0.3,False,False,False,False,0.0039,0.0271
242,BCResMax_LA_,0.6,False,False,False,False,0.0066,0.0273
128,LCNN_LA_,0.6,False,False,False,False,0.0082,0.0274
